# imports

In [ ]:
import pysubgroup as ps
import pandas as pd
import numpy as np
import import_ipynb
import os
import csv
import random
import pickle
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
aggregated_data_all = pd.read_pickle('AggregatedDataALL.pkl')
aggregated_data_usa = pd.read_pickle('AggregatedDataUSA.pkl')
aggregated_data_eu_apa = pd.read_pickle('AggregatedDataEUandAPA.pkl')
aggregated_data_eu = pd.read_pickle('AggregatedDataEU.pkl')
aggregated_data_apa = pd.read_pickle('AggregatedDataAPA.pkl')


# function imports

In [ ]:
from functions_definitions import resample_array                         # resamples an array to a given size ; resample_array(arr, fixed_length)
from functions_definitions import run_sd_array_magnitude                 # runs sg discovery once, wth an array target and magnitude and sign QF ; returns best result; used in peeling_magnitude
from functions_definitions import parse_array_from_string                #transforms a string into an array ; parse_array_from_string(string)
from functions_definitions import get_attr_array                         # returns the attributes of a discovered subgroup ; peeling_magnitude
from functions_definitions import append_and_cut_array                   # appends a discovered subgroup to the results and removes items from the discovery df; used in peeling_magnitude
from functions_definitions import peeling_magnitude                      # runs the entire disjoint subgroup discovery. explained below ;  peeling_magnitude(dataframe, "target column", alpha, minimum sg size)
from functions_definitions import get_sg_dfs                             # creates dataframes with the items from the discovered subgroups ; get_sg_dfs(aggregated_df, disjoint_sg_df)
from functions_definitions import get_sg_dfs_indexed                     # creates indexed dataframes with the items from the discovered subgroups ; get_sg_dfs_indexed(aggregated_df, disjoint_sg_df)
from functions_definitions import compute_avg_turnover_per_year          # computes turnover per semester given order data df
from functions_definitions import compute_timespans                      # computes the average time active of given subgroup and other product timespan metrics. made for period 2021-2024. modify in functions_definitions.ipynb if needed

# Run Disjoint Subgroup Discovery

First, we select only the relevant features for discovery from each dataframe. Then, the peeling_magnitude function runs the entire disjoint subgroup discovery. The quality function used in the backend by peeling_magnitude considers both the magnitude and sign of price elasticity. 

The parameters of the peeling_magnitude function are:

peeling_magnitude(dataframe, "target column", alpha, minimum sg size)

In this run, alpha is set at 0.5, giving equal importance to magnitude and sign. If you want to tweak this importance ratio, you can set alpha to any other value between 0 and 1. The higher the alpha, the more importance sign consistency has. When alpha = 1, only sign consistency matters. When alpha = 0, only magnitude matters.
 
Minimum sg size is set to 100, to ensure reliable prediction in the following steps. It is recommended not to decrease it if you will use these results for prediction later on. 

In [ ]:
# You can remove features from the analysis by removing them from the list below
discovery_data_usa = aggregated_data_usa[[ 'MeanSalesPrice', 'MeanCostPrice', 'MeanManualDiscount', 'MeanSupplierLeadtime', 'PriceCategory', 'PercentEU', 'PercentUSA', 'PercentAPA', 'VIEngineered', 'SparePartsCategory', 'TechnicalClassification', 'OEM', 'Level1', 'MeanProfit', 'MeanOrderQty', 'ItemsInOrder', 'DiscountPercent', 'Stocked', 'ClippedInflationTimeAdjPriceElasticity']]
discovery_data_apa = aggregated_data_apa[[ 'MeanSalesPrice', 'MeanCostPrice', 'MeanManualDiscount', 'MeanSupplierLeadtime', 'PriceCategory', 'PercentEU', 'PercentUSA', 'PercentAPA', 'VIEngineered', 'SparePartsCategory', 'TechnicalClassification', 'OEM', 'Level1', 'MeanProfit', 'MeanOrderQty', 'ItemsInOrder', 'DiscountPercent', 'Stocked', 'ClippedInflationTimeAdjPriceElasticity']]
discovery_data_eu_apa = aggregated_data_eu_apa[[ 'MeanSalesPrice', 'MeanCostPrice', 'MeanManualDiscount', 'MeanSupplierLeadtime', 'PriceCategory', 'PercentEU', 'PercentUSA', 'PercentAPA', 'VIEngineered', 'SparePartsCategory', 'TechnicalClassification', 'OEM', 'Level1', 'MeanProfit', 'MeanOrderQty', 'ItemsInOrder', 'DiscountPercent', 'Stocked', 'ClippedInflationTimeAdjPriceElasticity']]
discovery_data_eu = aggregated_data_eu[[ 'MeanSalesPrice', 'MeanCostPrice', 'MeanManualDiscount', 'MeanSupplierLeadtime', 'PriceCategory', 'PercentEU', 'PercentUSA', 'PercentAPA', 'VIEngineered', 'SparePartsCategory', 'TechnicalClassification', 'OEM', 'Level1', 'MeanProfit', 'MeanOrderQty', 'ItemsInOrder', 'DiscountPercent', 'Stocked', 'ClippedInflationTimeAdjPriceElasticity']]

In [ ]:
disjoint_sg_usa = peeling_magnitude(discovery_data_usa, "ClippedInflationTimeAdjPriceElasticity",  0.5, 100)

In [ ]:
disjoint_sg_usa_df= pd.DataFrame([s.to_dict() for s in disjoint_sg_usa])
disjoint_sg_usa_df.head()

In [ ]:
disjoint_sg_apa = peeling_magnitude(discovery_data_apa, "ClippedInflationTimeAdjPriceElasticity",  0.5, 100)

In [ ]:
disjoint_sg_apa_df= pd.DataFrame([s.to_dict() for s in disjoint_sg_apa])
disjoint_sg_apa_df

In [ ]:
disjoint_sg_eu = peeling_magnitude(discovery_data_eu, "ClippedInflationTimeAdjPriceElasticity",  0.5, 100)

In [ ]:
disjoint_sg_eu_df= pd.DataFrame([s.to_dict() for s in disjoint_sg_eu])
disjoint_sg_eu_df

In [ ]:
disjoint_sg_eu_apa = peeling_magnitude(discovery_data_eu_apa, "ClippedInflationTimeAdjPriceElasticity",  0.5, 100)

In [ ]:
disjoint_sg_eu_apa_df= pd.DataFrame([s.to_dict() for s in disjoint_sg_eu_apa])
disjoint_sg_eu_apa_df

# save disjoint results

In [ ]:
disjoint_sg_eu_df.to_csv("ResultsSgDisEU.csv", index = False)

In [ ]:
disjoint_sg_apa_df.to_csv("ResultsSgDisAPA.csv", index=False)

In [ ]:
disjoint_sg_usa_df.to_csv("ResultsSgDisUSA.csv", index=False)

In [ ]:
disjoint_sg_eu_apa_df.to_csv("ResultsSgDisEUandAPA.csv", index = False)

The results saved above are the raw results. Below, we will add additional information and save in an Excel format for easy inspection.

# Add information for Excel

We start by removing unnecessary columns and check that the dataframes are left with columns: quality, subgroup, size_sg, and centroid_sg

In [ ]:
short_disjoint_results_eu = disjoint_sg_eu_df.iloc[:, :8]
short_disjoint_results_apa = disjoint_sg_apa_df.iloc[:, :8]
short_disjoint_results_usa = disjoint_sg_usa_df.iloc[:, :8]
short_disjoint_results_eu_apa = disjoint_sg_eu_apa_df.iloc[:, :8]
short_disjoint_results_eu = short_disjoint_results_eu.drop(columns=['size_cover_all', 'covered_not_in_sg', 'size_dataset', 'centroid_dataset'])
short_disjoint_results_apa = short_disjoint_results_apa.drop(columns=['size_cover_all', 'covered_not_in_sg', 'size_dataset', 'centroid_dataset'])
short_disjoint_results_usa = short_disjoint_results_usa.drop(columns=['size_cover_all', 'covered_not_in_sg', 'size_dataset', 'centroid_dataset'])
short_disjoint_results_eu_apa = short_disjoint_results_eu_apa.drop(columns=['size_cover_all', 'covered_not_in_sg', 'size_dataset', 'centroid_dataset'])

In [ ]:
short_disjoint_results_eu

In [ ]:
short_disjoint_results_apa

In [ ]:
short_disjoint_results_usa

In [ ]:
short_disjoint_results_eu_apa

In [ ]:
short_disjoint_results_eu["Index"] = range(len(short_disjoint_results_eu))

In [ ]:
short_disjoint_results_apa["Index"] = range(len(short_disjoint_results_apa))

In [ ]:
short_disjoint_results_usa["Index"] = range(len(short_disjoint_results_usa))

In [ ]:
short_disjoint_results_eu_apa["Index"] = range(len(short_disjoint_results_eu_apa))

In [ ]:
eu_apa_subgroup_dfs_dict_indexed =  get_sg_dfs_indexed(aggregated_data_eu_apa, short_disjoint_results_eu_apa)

In [ ]:
eu_subgroup_dfs_dict_indexed =  get_sg_dfs_indexed(aggregated_data_eu, short_disjoint_results_eu)

In [ ]:
apa_subgroup_dfs_dict_indexed =  get_sg_dfs_indexed(aggregated_data_apa, short_disjoint_results_apa)

In [ ]:
usa_subgroup_dfs_dict_indexed =  get_sg_dfs_indexed(aggregated_data_usa, short_disjoint_results_usa)

In [ ]:
for index, df in apa_subgroup_dfs_dict_indexed.items():
    globals()[f"apa_items_sg_{index}"] = df["ItemNumber"].tolist()

In [ ]:
for index, df in usa_subgroup_dfs_dict_indexed.items():
    globals()[f"usa_items_sg_{index}"] = df["ItemNumber"].tolist()

In [ ]:
for index, df in eu_subgroup_dfs_dict_indexed.items():
    globals()[f"eu_items_sg_{index}"] = df["ItemNumber"].tolist()

In [ ]:
for index, df in eu_apa_subgroup_dfs_dict_indexed.items():
    globals()[f"eu_apa_items_sg_{index}"] = df["ItemNumber"].tolist()

In [ ]:
# Create a dictionary mapping from Index to the corresponding eu_items_sg_{index} list
eu_item_dict = {
    index: df["ItemNumber"].tolist()
    for index, df in eu_subgroup_dfs_dict_indexed.items()
}

# Add the column based on Index
short_disjoint_results_eu["all_items"] = short_disjoint_results_eu["Index"].apply(lambda idx: eu_item_dict.get(idx, []))

In [ ]:
# Create a dictionary mapping from Index to the corresponding apa_items_sg_{index} list
apa_item_dict = {
    index: df["ItemNumber"].tolist()
    for index, df in apa_subgroup_dfs_dict_indexed.items()
}

# Add the column based on Index
short_disjoint_results_apa["all_items"] = short_disjoint_results_apa["Index"].apply(lambda idx: apa_item_dict.get(idx, []))

In [ ]:
# Create a dictionary mapping from Index to the corresponding eu_apa_items_sg_{index} list
eu_apa_item_dict = {
    index: df["ItemNumber"].tolist()
    for index, df in eu_apa_subgroup_dfs_dict_indexed.items()
}

# Add the column based on Index
short_disjoint_results_eu_apa["all_items"] = short_disjoint_results_eu_apa["Index"].apply(lambda idx: eu_apa_item_dict.get(idx, []))

In [ ]:
# Create a dictionary mapping from Index to the corresponding usa_items_sg_{index} list
usa_item_dict = {
    index: df["ItemNumber"].tolist()
    for index, df in usa_subgroup_dfs_dict_indexed.items()
}

# Add the column based on Index
short_disjoint_results_usa["all_items"] = short_disjoint_results_usa["Index"].apply(lambda idx: usa_item_dict.get(idx, []))

In [ ]:

short_disjoint_results_eu["example_items"] = short_disjoint_results_eu["all_items"].apply(
    lambda items: random.sample(items, min(10, len(items)))
)

In [ ]:

short_disjoint_results_apa["example_items"] = short_disjoint_results_apa["all_items"].apply(
    lambda items: random.sample(items, min(10, len(items)))
)

In [ ]:

short_disjoint_results_usa["example_items"] = short_disjoint_results_usa["all_items"].apply(
    lambda items: random.sample(items, min(10, len(items)))
)

In [ ]:

short_disjoint_results_eu_apa["example_items"] = short_disjoint_results_eu_apa["all_items"].apply(
    lambda items: random.sample(items, min(10, len(items)))
)

In [ ]:
with open("eu_subgroup_dfs_dict_indexed.pkl", "wb") as f:
    pickle.dump(eu_subgroup_dfs_dict_indexed, f)

In [ ]:
with open("usa_subgroup_dfs_dict_indexed.pkl", "wb") as f:
    pickle.dump(usa_subgroup_dfs_dict_indexed, f)

In [ ]:
with open("apa_subgroup_dfs_dict_indexed.pkl", "wb") as f:
    pickle.dump(apa_subgroup_dfs_dict_indexed, f)

# get turnovers per semester


In [ ]:
wide_order_data_eu = pd.read_pickle('PreparedOrderDataEU.pkl')
wide_order_data_eu_apa = pd.read_pickle('PreparedOrderDataEUandAPA.pkl')
wide_order_data_usa = pd.read_pickle('PreparedOrderDataUSA.pkl')
wide_order_data_apa = pd.read_pickle('PreparedOrderDataAPA.pkl')

In [ ]:
wide_order_data_usa['OrderYear'] = wide_order_data_usa['OrderDate'].dt.year
wide_order_data_usa['Semester'] = wide_order_data_usa['OrderMonth'].apply(lambda x: 1 if x <= 6 else 2)
wide_order_data_usa['Turnover'] = wide_order_data_usa['PriceSalesUoMEUR'] * wide_order_data_usa['OrderQuantity']

In [ ]:
wide_order_data_apa['OrderYear'] = wide_order_data_apa['OrderDate'].dt.year
wide_order_data_apa['Semester'] = wide_order_data_apa['OrderMonth'].apply(lambda x: 1 if x <= 6 else 2)
wide_order_data_apa['Turnover'] = wide_order_data_apa['PriceSalesUoMEUR'] * wide_order_data_apa['OrderQuantity']

In [ ]:
wide_order_data_eu['OrderYear'] = wide_order_data_eu['OrderDate'].dt.year
wide_order_data_eu['Semester'] = wide_order_data_eu['OrderMonth'].apply(lambda x: 1 if x <= 6 else 2)
wide_order_data_eu['Turnover'] = wide_order_data_eu['PriceSalesUoMEUR'] * wide_order_data_eu['OrderQuantity']

In [ ]:
wide_order_data_eu_apa['OrderYear'] = wide_order_data_eu_apa['OrderDate'].dt.year
wide_order_data_eu_apa['Semester'] = wide_order_data_eu_apa['OrderMonth'].apply(lambda x: 1 if x <= 6 else 2)
wide_order_data_eu_apa['Turnover'] = wide_order_data_eu_apa['PriceSalesUoMEU_apaR'] * wide_order_data_eu_apa['OrderQuantity']

In [ ]:
long_disjoint_results_eu = short_disjoint_results_eu.copy()

for sg_num, item_list in eu_item_dict.items():
    # Filter order data for current subgroup
    order_data_eu_sg = wide_order_data_eu[wide_order_data_eu['ItemNumber'].isin(item_list)]

    # Compute turnover and timespan-related info
    avg_turnover_array = compute_avg_turnover_per_year(order_data_eu_sg)
    avg_turnover_array_string = ', '.join(map(str, avg_turnover_array))
    mean_timespan, items_introduced_2021_count, items_introduced_2022_count, items_introduced_2023_count, items_introduced_2024_count = compute_timespans(order_data_eu_sg)

    # Update the corresponding row in the results table
    long_disjoint_results_eu.loc[long_disjoint_results_eu['Index'] == sg_num, 'MeanTimespan'] = mean_timespan
    long_disjoint_results_eu.loc[long_disjoint_results_eu['Index'] == sg_num, 'NoItemsIntroduced2021'] = items_introduced_2021_count
    long_disjoint_results_eu.loc[long_disjoint_results_eu['Index'] == sg_num, 'NoItemsIntroduced2022'] = items_introduced_2022_count
    long_disjoint_results_eu.loc[long_disjoint_results_eu['Index'] == sg_num, 'NoItemsIntroduced2023'] = items_introduced_2023_count
    long_disjoint_results_eu.loc[long_disjoint_results_eu['Index'] == sg_num, 'NoItemsIntroduced2024'] = items_introduced_2024_count
    long_disjoint_results_eu.loc[long_disjoint_results_eu['Index'] == sg_num, 'AvgTurnover'] = avg_turnover_array_string

    # Optional: Logging
    print(f"SG {sg_num}: Mean Turnover {avg_turnover_array}")
    print(f"SG {sg_num}: Mean timespan: {mean_timespan}, Items introduced in 2021: {items_introduced_2021_count}, Items introduced in 2022: {items_introduced_2022_count}, Items introduced in 2023: {items_introduced_2023_count}, Items introduced in 2024: {items_introduced_2024_count}")


In [ ]:
long_disjoint_results_eu.head()

In [ ]:
long_disjoint_results_usa = short_disjoint_results_usa.copy()

for sg_num, item_list in usa_item_dict.items():
    # Filter order data for current subgroup
    order_data_usa_sg = wide_order_data_usa[wide_order_data_usa['ItemNumber'].isin(item_list)]

    # Compute turnover and timespan-related info
    avg_turnover_array = compute_avg_turnover_per_year(order_data_usa_sg)
    avg_turnover_array_string = ', '.join(map(str, avg_turnover_array))
    mean_timespan, items_introduced_2021_count, items_introduced_2022_count, items_introduced_2023_count, items_introduced_2024_count = compute_timespans(order_data_usa_sg)

    # Update the corresponding row in the results table
    long_disjoint_results_usa.loc[long_disjoint_results_usa['Index'] == sg_num, 'MeanTimespan'] = mean_timespan
    long_disjoint_results_usa.loc[long_disjoint_results_usa['Index'] == sg_num, 'NoItemsIntroduced2021'] = items_introduced_2021_count
    long_disjoint_results_usa.loc[long_disjoint_results_usa['Index'] == sg_num, 'NoItemsIntroduced2022'] = items_introduced_2022_count
    long_disjoint_results_usa.loc[long_disjoint_results_usa['Index'] == sg_num, 'NoItemsIntroduced2023'] = items_introduced_2023_count
    long_disjoint_results_usa.loc[long_disjoint_results_usa['Index'] == sg_num, 'NoItemsIntroduced2024'] = items_introduced_2024_count
    long_disjoint_results_usa.loc[long_disjoint_results_usa['Index'] == sg_num, 'AvgTurnover'] = avg_turnover_array_string

    # Optional: Logging
    print(f"SG {sg_num}: Mean Turnover {avg_turnover_array}")
    print(f"SG {sg_num}: Mean timespan: {mean_timespan}, Items introduced in 2021: {items_introduced_2021_count}, Items introduced in 2022: {items_introduced_2022_count}, Items introduced in 2023: {items_introduced_2023_count}, Items introduced in 2024: {items_introduced_2024_count}")


In [ ]:
long_disjoint_results_apa = short_disjoint_results_apa.copy()

for sg_num, item_list in apa_item_dict.items():
    # Filter order data for current subgroup
    order_data_apa_sg = wide_order_data_apa[wide_order_data_apa['ItemNumber'].isin(item_list)]

    # Compute turnover and timespan-related info
    avg_turnover_array = compute_avg_turnover_per_year(order_data_apa_sg)
    avg_turnover_array_string = ', '.join(map(str, avg_turnover_array))
    mean_timespan, items_introduced_2021_count, items_introduced_2022_count, items_introduced_2023_count, items_introduced_2024_count = compute_timespans(order_data_apa_sg)

    # Update the corresponding row in the results table
    long_disjoint_results_apa.loc[long_disjoint_results_apa['Index'] == sg_num, 'MeanTimespan'] = mean_timespan
    long_disjoint_results_apa.loc[long_disjoint_results_apa['Index'] == sg_num, 'NoItemsIntroduced2021'] = items_introduced_2021_count
    long_disjoint_results_apa.loc[long_disjoint_results_apa['Index'] == sg_num, 'NoItemsIntroduced2022'] = items_introduced_2022_count
    long_disjoint_results_apa.loc[long_disjoint_results_apa['Index'] == sg_num, 'NoItemsIntroduced2023'] = items_introduced_2023_count
    long_disjoint_results_apa.loc[long_disjoint_results_apa['Index'] == sg_num, 'NoItemsIntroduced2024'] = items_introduced_2024_count
    long_disjoint_results_apa.loc[long_disjoint_results_apa['Index'] == sg_num, 'AvgTurnover'] = avg_turnover_array_string

    # Optional: Logging
    print(f"SG {sg_num}: Mean Turnover {avg_turnover_array}")
    print(f"SG {sg_num}: Mean timespan: {mean_timespan}, Items introduced in 2021: {items_introduced_2021_count}, Items introduced in 2022: {items_introduced_2022_count}, Items introduced in 2023: {items_introduced_2023_count}, Items introduced in 2024: {items_introduced_2024_count}")


In [ ]:
long_disjoint_results_eu_apa = short_disjoint_results_eu_apa.copy()

for sg_num, item_list in eu_apa_item_dict.items():
    # Filter order data for current subgroup
    order_data_eu_apa_sg = wide_order_data_eu_apa[wide_order_data_eu_apa['ItemNumber'].isin(item_list)]

    # Compute turnover and timespan-related info
    avg_turnover_array = compute_avg_turnover_per_year(order_data_eu_apa_sg)
    avg_turnover_array_string = ', '.join(map(str, avg_turnover_array))
    mean_timespan, items_introduced_2021_count, items_introduced_2022_count, items_introduced_2023_count, items_introduced_2024_count = compute_timespans(order_data_eu_apa_sg)

    # Update the corresponding row in the results table
    long_disjoint_results_eu_apa.loc[long_disjoint_results_eu_apa['Index'] == sg_num, 'MeanTimespan'] = mean_timespan
    long_disjoint_results_eu_apa.loc[long_disjoint_results_eu_apa['Index'] == sg_num, 'NoItemsIntroduced2021'] = items_introduced_2021_count
    long_disjoint_results_eu_apa.loc[long_disjoint_results_eu_apa['Index'] == sg_num, 'NoItemsIntroduced2022'] = items_introduced_2022_count
    long_disjoint_results_eu_apa.loc[long_disjoint_results_eu_apa['Index'] == sg_num, 'NoItemsIntroduced2023'] = items_introduced_2023_count
    long_disjoint_results_eu_apa.loc[long_disjoint_results_eu_apa['Index'] == sg_num, 'NoItemsIntroduced2024'] = items_introduced_2024_count
    long_disjoint_results_eu_apa.loc[long_disjoint_results_eu_apa['Index'] == sg_num, 'AvgTurnover'] = avg_turnover_array_string

    # Optional: Logging
    print(f"SG {sg_num}: Mean Turnover {avg_turnover_array}")
    print(f"SG {sg_num}: Mean timespan: {mean_timespan}, Items introduced in 2021: {items_introduced_2021_count}, Items introduced in 2022: {items_introduced_2022_count}, Items introduced in 2023: {items_introduced_2023_count}, Items introduced in 2024: {items_introduced_2024_count}")


# save results with turnover

In [ ]:
long_disjoint_results_eu.to_excel("InfoResultsSgDisEU.xlsx", index=False)
long_disjoint_results_apa.to_excel("InfoResultsSgDisAPA.xlsx", index=False)
long_disjoint_results_usa.to_excel("InfoResultsSgDisUSA.xlsx", index=False)
long_disjoint_results_eu_apa.to_excel("InfoResultsSgDisEUandAPA.xlsx", index=False)